https://www.kaggle.com/misrakahmed/vegetable-image-dataset

데이터를 분류하는 데에 여러 모델들을 사용해 보고 성능, 학습 시간 비교

In [1]:
# 필수 코드

import torch
import torchvision
import visdom

vis = visdom.Visdom()
vis.close(env='main')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.random.manual_seed(0)
if device == 'cuda':
    torch.cuda.manual_seed_all(0)

Setting up a new session...


In [2]:
# hyperparameters
batch_size = 1800
learning_rate = 1e-3
epoch = 50

image_size = 56
class_num = 15
class_name = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

In [3]:
# train data loader를 반환함
def get_train_data_loaders(resize = False, rotate = False):
    transforms = []

    if rotate:
        transforms.append(torchvision.transforms.transforms.RandomRotation(360))

    if resize:
        transforms.append(torchvision.transforms.RandomResizedCrop((image_size, image_size)))
        
    transforms.append(torchvision.transforms.Resize((image_size, image_size)))
    transforms.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(transforms)

    train_data = torchvision.datasets.ImageFolder(root='train', transform=transform)
    train_data_loader =torch.utils.data.DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True, drop_last=True)

    return train_data_loader

In [4]:
# test data loader와 validation data loader를 반환함
def get_test_validation_data_loaders():
    transform = torchvision.transforms.Compose([torchvision.transforms.Resize((image_size, image_size)), torchvision.transforms.ToTensor()])
    test_data = torchvision.datasets.ImageFolder(root='test', transform=transform)
    validation_data = torchvision.datasets.ImageFolder(root='validation', transform=transform)

    test_data_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=batch_size, shuffle=False, drop_last=True)
    validation_data_loader =torch.utils.data.DataLoader(dataset = validation_data, batch_size=batch_size, shuffle=False, drop_last=True)

    return test_data_loader, validation_data_loader

In [5]:
def test_model(model, data_loader):
    with torch.no_grad():
        accuracy = 0
        for X, Y in data_loader:
            X = X.to(device)
            Y = Y.to(device)
            prediction = model(X)
            correct_prediction = torch.argmax(prediction, dim=1) == Y
            accuracy += correct_prediction.float().mean()
        accuracy /= len(data_loader)
        print("accuracy : {:.4f}%".format(accuracy.item() * 100))
        del X
        del Y
        torch.cuda.empty_cache()

    return accuracy.item() * 100

In [6]:
#model을 data_loader로 epoch만큼 학습함, print_loss가 True이면 trainloss를 출력함
def train_model(model, data_loader, test_loader, opt, epochs, print_loss=False):

    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = opt(model.parameters(), lr=learning_rate)
    total_batch = len(data_loader)

    if print_loss:
        loss_plot = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True), env='main')
        acc_plot = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='accuracy_tracker', legend=['accuracy'], showlegend=True), env='main')

    for epoch in range(epochs):
        avg_cost = 0.0
        for X, Y in data_loader:
            X = X.to(device)
            Y = Y.to(device)
            output = model(X)
            loss = criterion(output, Y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_cost += loss/total_batch
        
        del X
        del Y
        torch.cuda.empty_cache()
        
        if print_loss:
            print('[EPOCH:{:3d}] cost:{:.5f}'.format(epoch + 1, avg_cost))
            vis.line(X=torch.Tensor([epoch]), Y=torch.Tensor([avg_cost]), win=loss_plot, update="append")
            acc = test_model(model, test_loader)
            vis.line(X=torch.Tensor([epoch]), Y=torch.Tensor([acc]), win=acc_plot, update="append")

In [7]:
# Fully connected model
class FCmodel(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        layer_list = []
        in_size = config[0]
        for i in range(1, len(config) - 1):
            out_size = config[i]
            layer_list.append(torch.nn.Linear(in_size, out_size, bias=True))
            layer_list.append(torch.nn.ReLU())
            in_size = out_size
        
        out_size = config[-1]
        layer_list.append(torch.nn.Linear(in_size, out_size, bias=True))

        self.layers = torch.nn.Sequential(*layer_list)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.layers(x)

In [8]:
# CNN model
class CNN(torch.nn.Module):
    def __init__(self, config, config_fc = None):
        super().__init__()
        layer_list = []
        channel_in_size = 3
        in_size = image_size
        for cont in config:
            if cont == 'M':
                layer_list.append(torch.nn.MaxPool2d(2))
                in_size = in_size // 2
            else:
                channel, kernel_size, padding = cont
                layer_list.append(torch.nn.Conv2d(channel_in_size, channel, kernel_size, padding=padding))
                layer_list.append(torch.nn.ReLU())
                channel_in_size = channel
                in_size = in_size - kernel_size + 2 * padding + 1
        
        self.layers = torch.nn.Sequential(*layer_list)

        if config_fc == None:
            self.fc = FCmodel([in_size * in_size * channel_in_size, 128, 32, class_num])
        else:
            self.fc = FCmodel(config_fc)
        
    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [9]:
# VGG model
class VGG(torch.nn.Module):
    def __init__(self, config, init_weights=True):
        super(VGG, self).__init__()
        
        channel_in_size = 3
        layer_list = []
        for cont in config:
            if cont == 'M':
                layer_list.append(torch.nn.MaxPool2d(2))
            else:
                layer_list.append(torch.nn.Conv2d(channel_in_size, cont, 3, padding=1))
                layer_list.append(torch.nn.ReLU())
                channel_in_size = cont
        
        self.features = torch.nn.Sequential(*layer_list)

        
        self.avgpool = torch.nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(channel_in_size * 7 * 7, 512),
            torch.nn.ReLU(True),
            torch.nn.Dropout(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(True),
            torch.nn.Dropout(),
            torch.nn.Linear(128, class_num),
        )#FC layer
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) #Convolution 
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) #
        x = self.classifier(x) #FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.BatchNorm2d):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.Linear):
                torch.nn.init.normal_(m.weight, 0, 0.01)
                torch.nn.init.constant_(m.bias, 0)

In [10]:
class InceptionA(torch.nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        
        self.branch_pool = torch.nn.Conv2d(in_channels, 24, kernel_size=1)
        
        self.branch1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16,24,kernel_size=5, padding=2)
        
        self.branch3x3dbl_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        
        branch_pool = torch.nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1,  branch5x5, branch3x3dbl, branch_pool]
        
        return torch.cat(outputs, 1)
    
class InceptionNet(torch.nn.Module):
    def __init__(self, activation = 'ReLU'):
        super(InceptionNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(3,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88,20,kernel_size=5)
        self.conv3 = torch.nn.Conv2d(88,30,kernel_size=5)
        
        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)
        self.incept3 = InceptionA(in_channels=30)
        af = {'ReLU' : torch.nn.ReLU(), 'Sigmoid' : torch.nn.Sigmoid(), 'LeakyReLU' : torch.nn.LeakyReLU(), 'tanh' : torch.nn.Tanh()}
        self.activation = af[activation]

        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(792,15)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.activation(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = self.activation(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = self.activation(self.mp(self.conv3(x)))
        x = self.incept3(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

In [11]:
#model1 = FCmodel([image_size*image_size*3, 2048, 512, 128, class_num]).to(device)
#model2 = CNN([(32, 3, 0), 'M', (64, 3, 0), 'M', (128, 3, 0), 'M']).to(device)
#model3 = VGG([64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512]).to(device)
# model4 = InceptionNet('ReLU').to(device)
# print(model4)
train_loader = get_train_data_loaders()
test_loader, valid_loader = get_test_validation_data_loaders()

In [12]:
# train_model(model4, train_loader, test_loader, torch.optim.Adam, epoch, print_loss=True)

In [13]:
# torch.save(model4.state_dict(), './model/inception_relu_50')

In [14]:
# del model4
# torch.cuda.empty_cache()

# model4 = InceptionNet('Sigmoid').to(device)
# train_model(model4, train_loader, test_loader, torch.optim.Adam, epoch, print_loss=True)

In [15]:
# torch.save(model4.state_dict(), './model/inception_Sigmoid_50')

In [16]:
# del model4
# torch.cuda.empty_cache()

# model4 = InceptionNet('LeakyReLU').to(device)
# train_model(model4, train_loader, test_loader, torch.optim.Adam, epoch, print_loss=True)
# torch.save(model4.state_dict(), './model/inception_LeakyReLU_50')

In [17]:
# del model4
# torch.cuda.empty_cache()

# model4 = InceptionNet('tanh').to(device)
# train_model(model4, train_loader, test_loader, torch.optim.Adam, epoch, print_loss=True)
# torch.save(model4.state_dict(), './model/inception_tanh_50')

In [18]:
torch.cuda.empty_cache()

model4 = InceptionNet('ReLU').to(device)
train_model(model4, train_loader, test_loader, torch.optim.SGD, epoch, print_loss=True)
torch.save(model4.state_dict(), './model/inception_SGD_50')

[EPOCH:  1] cost:2.70881
accuracy : 11.1111%
[EPOCH:  2] cost:2.70877
accuracy : 11.1111%
[EPOCH:  3] cost:2.70873
accuracy : 11.1111%
[EPOCH:  4] cost:2.70886
accuracy : 11.1111%
[EPOCH:  5] cost:2.70880
accuracy : 11.1111%
[EPOCH:  6] cost:2.70869
accuracy : 11.1111%
[EPOCH:  7] cost:2.70879
accuracy : 11.1111%
[EPOCH:  8] cost:2.70890
accuracy : 11.1111%
[EPOCH:  9] cost:2.70871
accuracy : 11.1111%
[EPOCH: 10] cost:2.70872
accuracy : 11.1111%
[EPOCH: 11] cost:2.70884
accuracy : 11.1111%
[EPOCH: 12] cost:2.70879
accuracy : 11.1111%
[EPOCH: 13] cost:2.70870
accuracy : 11.1111%
[EPOCH: 14] cost:2.70877
accuracy : 11.1111%
[EPOCH: 15] cost:2.70871
accuracy : 11.1111%
[EPOCH: 16] cost:2.70874
accuracy : 11.1111%
[EPOCH: 17] cost:2.70869
accuracy : 11.1111%
[EPOCH: 18] cost:2.70873
accuracy : 11.1111%
[EPOCH: 19] cost:2.70867
accuracy : 11.1111%
[EPOCH: 20] cost:2.70867
accuracy : 11.1111%
[EPOCH: 21] cost:2.70873
accuracy : 11.1111%
[EPOCH: 22] cost:2.70871
accuracy : 11.1111%
[EPOCH: 23

In [19]:
del model4
torch.cuda.empty_cache()

model4 = InceptionNet('ReLU').to(device)
train_model(model4, train_loader, test_loader, torch.optim.Adagrad, epoch, print_loss=True)
torch.save(model4.state_dict(), './model/inception_Adagrad_50')

[EPOCH:  1] cost:2.69407
accuracy : 17.2778%
[EPOCH:  2] cost:2.55275
accuracy : 3.1667%
[EPOCH:  3] cost:2.53134
accuracy : 9.8889%
[EPOCH:  4] cost:2.35331
accuracy : 7.3889%
[EPOCH:  5] cost:2.35185
accuracy : 11.8333%
[EPOCH:  6] cost:2.28350
accuracy : 11.2222%
[EPOCH:  7] cost:2.28197
accuracy : 13.6111%
[EPOCH:  8] cost:2.21946
accuracy : 16.7222%
[EPOCH:  9] cost:2.21716
accuracy : 23.7778%
[EPOCH: 10] cost:2.17881
accuracy : 17.2222%
[EPOCH: 11] cost:2.14687
accuracy : 19.6667%
[EPOCH: 12] cost:2.10660
accuracy : 20.3889%
[EPOCH: 13] cost:2.08002
accuracy : 19.5556%
[EPOCH: 14] cost:2.02597
accuracy : 26.7778%
[EPOCH: 15] cost:1.96330
accuracy : 30.3333%
[EPOCH: 16] cost:1.90296
accuracy : 31.7222%
[EPOCH: 17] cost:1.87523
accuracy : 33.0000%
[EPOCH: 18] cost:1.88884
accuracy : 33.5000%
[EPOCH: 19] cost:1.79524
accuracy : 35.5556%
[EPOCH: 20] cost:1.76236
accuracy : 36.0556%
[EPOCH: 21] cost:1.76050
accuracy : 31.3333%
[EPOCH: 22] cost:1.81905
accuracy : 36.3889%
[EPOCH: 23] c

In [20]:
del model4
torch.cuda.empty_cache()

model4 = InceptionNet('ReLU').to(device)
train_model(model4, train_loader, test_loader, torch.optim.RMSprop, epoch, print_loss=True)
torch.save(model4.state_dict(), './model/inception_RMSprop_50')

[EPOCH:  1] cost:33.22563
accuracy : 1.5000%
[EPOCH:  2] cost:2.86175
accuracy : 13.1111%
[EPOCH:  3] cost:2.64636
accuracy : 13.3889%
[EPOCH:  4] cost:2.56144
accuracy : 9.7778%
[EPOCH:  5] cost:2.49616
accuracy : 8.2222%
[EPOCH:  6] cost:2.50911
accuracy : 8.5000%
[EPOCH:  7] cost:2.47144
accuracy : 9.2222%
[EPOCH:  8] cost:2.44909
accuracy : 8.8333%
[EPOCH:  9] cost:2.44832
accuracy : 12.6667%
[EPOCH: 10] cost:2.41934
accuracy : 15.5556%
[EPOCH: 11] cost:2.40281
accuracy : 16.8333%
[EPOCH: 12] cost:2.40012
accuracy : 12.6667%
[EPOCH: 13] cost:2.37652
accuracy : 20.2222%
[EPOCH: 14] cost:2.35835
accuracy : 21.3333%
[EPOCH: 15] cost:2.33851
accuracy : 17.7778%
[EPOCH: 16] cost:2.35462
accuracy : 19.8333%
[EPOCH: 17] cost:2.32705
accuracy : 22.2778%
[EPOCH: 18] cost:2.37992
accuracy : 26.0556%
[EPOCH: 19] cost:2.27824
accuracy : 18.9444%
[EPOCH: 20] cost:2.32409
accuracy : 28.0000%
[EPOCH: 21] cost:2.24536
accuracy : 24.7222%
[EPOCH: 22] cost:2.23793
accuracy : 16.2778%
[EPOCH: 23] cos